In [1]:
import numpy as np
import itertools as iter

# Написать функции кодирования и декодирования для циклического кода порождающим многочленом 𝑔(𝑥) = 1+𝑥^2 +𝑥^3, исправляющего однократные ошибки и провести исследование этого кода для одно-, двух- и трёхкратных ошибок.

In [2]:
def create_G(n, m, g_x):
    G = np.zeros((m, n), dtype=int)
    for i in range(m):
        G[i, i: i + g_x.size] = g_x
    for i in range(1, m):
        for j in range(i):
            if G[j][i] == 1:
                G[j] ^= G[i]
    return G

In [3]:
def vec_to_num(v):
    return (np.array([2 ** i for i in range(v.size)]) * v).sum()

In [4]:
def find_remainder(bin_pol_num_1, bin_pol_num_2, n, m):
    bin_pol_num_2 <<= m
    for i in range(1, m + 1):
        bin_pol_num_2 >>= 1
        if bin_pol_num_1 >= 1 << (n - i):
            bin_pol_num_1 ^= bin_pol_num_2
    return bin_pol_num_1

In [5]:
def decode_word(w, m, t, g_x_num):
    n = w.size
    w_num = vec_to_num(w)
    w_num = find_remainder(w_num, g_x_num, n, m)
    if w_num == 0:
        return w.copy()[:m]
    k = 0
    while k > -n:
        if w_num < 1 << t and w_num % 2 == 1:  #
            break
        w_num <<= 1
        if w_num >= 1 << (n - m):
            w_num ^= g_x_num
        k -= 1
    w_fix = w.copy()
    while w_num != 0:
        if w_num % 2 == 1:
            w_fix[k] ^= 1
        k += 1
        w_num >>= 1
    return w_fix[:m]

In [6]:
n = 7
m = 4
t = 1
g_x = np.array([1, 0, 1, 1])
g_x_num = vec_to_num(g_x)
g_x_num

13

In [7]:
G = create_G(n, m, g_x)
G

array([[1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 1]])

In [8]:
word = np.array([1, 0, 0, 1])
v = np.dot(word, G) % 2
v

array([1, 0, 0, 1, 1, 1, 0])

In [9]:
err2 = np.zeros_like(v, dtype=int)
err2[1] = err2[2] = 1
err3 = err2.copy()
err3[3] = 1

In [10]:
w_err2 = v ^ err2
w_err3 = v ^ err3
w_err1_array = [v.copy() ^ err1 for err1 in np.eye(n, dtype=int)]
w_err1_array

[array([0, 0, 0, 1, 1, 1, 0]),
 array([1, 1, 0, 1, 1, 1, 0]),
 array([1, 0, 1, 1, 1, 1, 0]),
 array([1, 0, 0, 0, 1, 1, 0]),
 array([1, 0, 0, 1, 0, 1, 0]),
 array([1, 0, 0, 1, 1, 0, 0]),
 array([1, 0, 0, 1, 1, 1, 1])]

In [11]:
dec_word = decode_word(v, m, t, g_x_num)
np.array_equal(dec_word, word)

True

In [12]:
for w in w_err1_array:
    dec_word = decode_word(w, m, t, g_x_num)
    print(np.array_equal(dec_word, word))

True
True
True
True
True
True
True


In [13]:
dec_word = decode_word(w_err2, m, t, g_x_num)
np.array_equal(dec_word, word)

False

In [14]:
dec_word = decode_word(w_err3, m, t, g_x_num)
np.array_equal(dec_word, word)

False

# Написать функции кодирования и декодирования для циклического кода (15,9) с порождающим многочленом 𝑔(𝑥) = 1+ 𝑥^3 +𝑥^4 +𝑥^5 +𝑥^6, исправляющего пакеты ошибок кратности 3 и провести исследование этого кода для пакетов ошибок длины 1, 2, 3 и 4.

In [15]:
n = 15
m = 9
t = 3
g_x = np.array([1, 1, 1, 1, 0, 0, 1])
g_x_num = vec_to_num(g_x)
g_x_num

79

In [16]:
G = create_G(n, m, g_x)
G

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1]])

In [17]:
word = np.array([1, 1, 1, 0, 0, 0, 0, 0, 0])
v = np.dot(word, G) % 2
v

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1])

In [18]:
err2 = np.zeros_like(v, dtype=int)
err2[1] = err2[2] = 1
err3 = err2.copy()
err3[3] = 1
err4 = np.zeros_like(v, dtype=int)
err4[6] = err4[7] = err4[9] = 1

In [19]:
w_err2 = v ^ err2
w_err3 = v ^ err3
w_err4 = v ^ err4
w_err1_array = [v.copy() ^ err1 for err1 in np.eye(n, dtype=int)]
w_err1_array

[array([0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0])]

In [20]:
dec_word = decode_word(v, m, t, g_x_num)
np.array_equal(dec_word, word)

True

In [21]:
for w in w_err1_array:
    dec_word = decode_word(w, m, t, g_x_num)
    print(np.array_equal(dec_word, word))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [22]:
dec_word = decode_word(w_err2, m, t, g_x_num)
np.array_equal(dec_word, word)

True

In [23]:
dec_word = decode_word(w_err3, m, t, g_x_num)
np.array_equal(dec_word, word)

True

In [24]:
dec_word = decode_word(w_err4, m, t, g_x_num)
np.array_equal(dec_word, word)

False